In [1]:
import os

import numpy as np
import pandas as pd

In [2]:
# read metadata
df = pd.read_csv('metadata_compiled.csv')
df.head()

,uuid,datetime,cough_detected,SNR,latitude,longitude,age,gender,respiratory_condition,fever_muscle_pain,...,quality_4,cough_type_4,dyspnea_4,wheezing_4,stridor_4,choking_4,congestion_4,nothing_4,diagnosis_4,severity_4
0,00014dcc-0f06-4c27-8c7b-737b18a2cf4c,2020-11-25T18:58:50.488301+00:00,0.0155,7.326171,48.9,2.4,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,00039425-7f3a-42aa-ac13-834aaa2b6b92,2020-04-13T21:30:59.801831+00:00,0.9609,16.151433,31.3,34.8,15.0,male,False,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0007c6f1-5441-40e6-9aaf-a761d8f2da3b,2020-10-18T15:38:38.205870+00:00,0.1643,16.217201,NaN,NaN,46.0,female,False,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0009eb28-d8be-4dc1-92bb-907e53bc5c7a,2020-04-12T04:02:18.159383+00:00,0.9301,20.146058,40.0,-75.1,34.0,male,True,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0012c608-33d0-4ef7-bde3-75a0b1a0024e,2020-04-15T01:03:59.029326+00:00,0.0482,0.000000,-16.5,-71.5,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
# filter data (cough)
df_cough_stat = df.loc[(df['cough_detected'] == 1)] # only get audio which was approved as 'cough_detected' by expert
df_cough_stat = df_cough_stat[['uuid', 'cough_detected', 'status', 'SNR']].dropna() # only take relevant column
df_cough_stat.reset_index(drop=True, inplace=True)
df_cough_stat.head()

,uuid,cough_detected,status,SNR
0,0029d048-898a-4c70-89c7-0815cdcf7391,1.0,symptomatic,9.624196
1,005b8518-03ba-4bf5-86d2-005541442357,1.0,healthy,17.428288
2,006d8d1c-2bf6-46a6-8ef2-1823898a4733,1.0,symptomatic,13.284361
3,008ba489-31ad-44d8-856b-fcf72369dc46,1.0,healthy,12.711530
4,008c1c9e-aeef-40c5-846c-24f1b964f884,1.0,symptomatic,55.824914


In [31]:
# filter data (non-cough)
# df_noncough_stat = df.loc[(df['cough_detected'] == 0)] # only get audio which was approved as not 'cough_detected' by expert
# df_noncough_stat = df_noncough_stat[['uuid', 'cough_detected', 'status', 'SNR']].dropna() # only take relevant column
# df_noncough_stat.reset_index(drop=True, inplace=True)
# df_noncough_stat.head()

In [4]:
# get the label distribution
df_cough_stat['status'].value_counts()

symptomatic    1035
healthy         766
COVID-19        699
Name: status, dtype: int64

In [5]:
# # get the label distribution
# df_noncough_stat['status'].value_counts()

In [6]:
# save filtered metadata to CSV
df_cough_stat.to_csv('metadata_filtered.csv', index=False)

In [13]:
def convert_audio(src_folder, dst_folder, audio_list):
    '''
    convert audio is done separately because of environment limitation, 
    only move the audio to the designated folder in this function.
    
    for src in filtered_raw/*.ogg; do ffmpeg -i "$src" -f wav -acodec pcm_s16le -ar 48000 
    -ac 1 filtered_wav/$(basename $src .ogg).wav; done;
    for src in filtered_raw/*.webm; do ffmpeg -i "$src" -f wav -acodec pcm_s16le -ar 48000 
    -ac 1 filtered_wav/$(basename $src .webm).wav; done;
    '''
    
    if not(os.path.exists(src_folder)) or not(os.listdir(src_folder)):
        print('Error: invalid source folder or source folder is empty')
        return
        
    if not(os.path.exists(dst_folder)):
        os.makedirs(dst_folder)
    
    for audio in audio_list:
        src_path = os.path.join(src_folder, audio)
        dst_path = os.path.join(dst_folder, audio)
        if os.path.isfile(src_path + '.webm'):
            pass
#             os.rename(src_path + '.webm', dst_path + '.webm')
        elif os.path.isfile(src_path + '.ogg'):
            pass
#             os.rename(src_path + '.ogg', dst_path + '.ogg')
        else:
            pass
#             print('Error: File not found {}'.format(audio))

In [15]:
# get WAV file from the filtered data
files_filtered = df_cough_stat.uuid.to_numpy()
convert_audio('public_dataset/', 'filtered/', files_filtered)